# 字

https://github.com/mike-fabian/ibus-table-chinese/blob/main/tables/

In [43]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
# https://github.com/mike-fabian/ibus-table-chinese/blob/main/tables/
df = pd.read_csv(
#     "data/cangjie5.txt",
    "data/cangjie-big.txt",
    comment="#",
    sep="\t",
    quoting=3,  # csv.QUOTE_NONE
#     skiprows=148,
    skiprows=160,
    skipfooter=2,
    keep_default_na=False,
    names=["cangjie", "hanzi", "noidea"],
)[["hanzi", "cangjie"]]

df_cj = df.groupby("hanzi").agg({"cangjie": lambda x: " ".join(x)})
df_cj.head(1000).tail()
# dupe = "hanzi"
# df[df[dupe].duplicated(keep=False)].sort_values(dupe).to_csv("cangjie_duplicated.csv", index=False)

/var/folders/q1/w0s_n4x93y30t1tf6m9x42rw0000gn/T/ipykernel_81627/1029852065.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(


,cangjie
hanzi,
㞖,sdb
㞗,sije
㞘,smso
㞙,sue
㞚,smjx


In [3]:
df = pd.read_csv(
    "data/wubi-haifeng86.UTF-8",
    comment="#",
    sep="\t",
    quoting=3,  # csv.QUOTE_NONE
    skiprows=0,
    skipfooter=0,
    keep_default_na=False,
    names=["wubi", "hanzi", "noidea"],
)[["hanzi", "wubi"]]

# not sure what len(hanzi)>1 means when it has a full stop, but just remove
df_wb = df[df["hanzi"].map(len).eq(1)].groupby("hanzi").agg({"wubi": lambda x: " ".join(x)})
df_wb.head(3000).tail()

,wubi
hanzi,
㓎,uvpc
㓏,umwk
㓐,ufwf
㓑,ufuf
㓒,uawn


In [4]:
df_dc = (
    pd.read_csv("data/ChineseCharacterMap.csv", keep_default_na=False, comment="#")
    .rename(columns={"字(Character)": "hanzi", "构件(Decomposition)": "components"})
    [["hanzi", "components"]]
    .set_index("hanzi")
)
df_dc.head()

,components
hanzi,
人,人
口,口
的,白勺
白,白
勹,勹


In [65]:
df_zi = df_cj.join(df_wb, how="outer").join(df_dc, how="outer")
# df_zi = df_zi.assign(traditionals=df_zi.index.map(lambda x: "".join(simp2trad[x])))

with open("../src/data/zi.json", "wt") as f:
    f.write(json.dumps(df_zi.replace(np.nan, None).reset_index().to_dict(orient="list"), ensure_ascii=False))

In [6]:
df_zi[df_zi["components"].notna()].head(20).tail(10)

,cangjie,wubi,components
hanzi,,,
㙯,gimmi,fwfc,埶云
㝉,jm,pgf,宀一
㡭,pvii,xxgn,丝
㣇,vhlb,xgrh,彑
㥯,bmsp,eavn,心𤔌
㦰,ooi,wwae wwai,戈人
㫃,yso,ywy,㫃
㱿,gbhne,fmgc,殳南
㳄,eno,iqwy,水欠


# 汉语水平考试

https://github.com/plaktos/hsk_csv

In [7]:
df_hsk = pd.concat(
    (
        pd.read_csv(f"data/hsk{i}.csv", names=["simplified", "pinyin", "meanings"])
        .assign(level=i)
    )
    for i in range(1,7)
)[["simplified", "meanings", "level"]]

# 词

https://github.com/marcanuy/cedict_utils

In [9]:
from cedict import CedictParser
import cedict
import importlib
importlib.reload(cedict)

parser = CedictParser(file_path="data/cedict_ts.u8")
entries = parser.parse()
entries[0].jyutping

''

In [10]:
df_ci = pd.DataFrame({
    "simplified": [e.simplified for e in entries],
    "traditional": [e.traditional for e in entries],
    "pinyin": [e.pinyin for e in entries],
#     "jyutping": [e.jyutping for e in entries],  # added support for cantonese dict but the data is trash
    "meanings": [e.meanings for e in entries],  # flattened meanings to store more easily in csv
})
df_ci.head()

,simplified,traditional,pinyin,meanings
0,%,%,pa1,percent (Tw)
1,2019冠状病毒病,2019冠狀病毒病,er4 ling2 yi1 jiu3 guan1 zhuang4 bing4 du2 bing4,"COVID-19, the coronavirus disease identified i..."
2,21三体综合症,21三體綜合症,er4 shi2 yi1 san1 ti3 zong1 he2 zheng4,trisomy; Down's syndrome
3,3C,3C,san1 C,"computers, communications, and consumer electr..."
4,3P,3P,san1 P,(slang) threesome


In [11]:
merged = df_hsk[["simplified", "meanings"]].merge(df_ci[["simplified", "pinyin"]], on="simplified", how="left")

In [12]:
df_missing = merged[merged["pinyin"].isna()]
df_missing = (
    df_missing
    .assign(traditional=["打籃球", "繫領帶", "紐扣儿", "致力於"])
    .assign(pinyin=["da3 lan2 qiu2", "ji4 ling3 dai4", "niu3 ko4 r5", "zhi4 li4 yu2"])
)
df_ci = pd.concat([df_ci, df_missing])

In [13]:
df_ci.tail()

,simplified,traditional,pinyin,meanings
122664,𰻝𰻝面,𰻞𰻞麵,biang2 biang2 mian4,"broad, belt-shaped noodles, popular in Shaanxi"
248,打篮球,打籃球,da3 lan2 qiu2,Play basketball
2023,系领带,繫領帶,ji4 ling3 dai4,to tie one's necktie
4365,纽扣儿,紐扣儿,niu3 ko4 r5,Buttons children
5466,致力于,致力於,zhi4 li4 yu2,Committed to


In [20]:
import pycantonese

# get rid of any words that can't be translated to cantonese
def mapper(trad):
    replacements = {
        "21": "二十一",
        "0": "零",
        "1": "一",
        "2": "二",
        "3": "三",
        "4": "四",
        "5": "五",
        "6": "六",
        "7": "七",
        "8": "八",
        "9": "九",
        "，": ",",  # for some reason the input data uses ',' for pinyin but '，' for hanzi
        "、": ",",  # sometimes a different comma
        "：": ":",  # and for colon
    }
    for k, v in replacements.items():
        trad = trad.replace(k, v)
    
    # note that pycantonese will actually translate words like 'K仔' into 'kei1 zai2'
    # which is correct but does not match the input cedict data
    jyut = pycantonese.characters_to_jyutping(trad)
    for in_, out in jyut:
        if out is None and not re.match(",|·|:|[a-zA-Z]+", in_):
            return None
        
    return " ".join(
        " ".join(re.findall("[a-zA-Z]+[0-9]", out))
        if out is not None# and not re.fullmatch("[a-zA-Z]+", in_)
        else " ".join(in_) for in_, out in jyut
    )

df_ci["jyutping"] = df_ci["traditional"].map(mapper)

In [21]:
df_ci[df_ci["jyutping"].isna()]
# df_ci[df_ci["pinyin"].str.contains("xx")]

,simplified,traditional,pinyin,meanings,jyutping
113,π日,π日,Pai4 ri4,"World Pi Day, a celebration of the mathematica...",None
114,□,□,biang4,"(Tw) (coll.) cool/awesome/(etymologically, a c...",None
115,□,□,biu1,(onom.) pew! (sound of a bullet fired from a g...,None
116,□,□,ging1,uptight; obstinate; to awkwardly force oneself...,None
118,⺮,⺮,zhu2,"""bamboo"" radical in Chinese characters (Kangxi...",None
119,々,々,xx5,iteration mark (used to represent a duplicated...,None
121,〡,〡,yi1,numeral 1 in the Suzhou numeral system 蘇州碼子|苏州...,None
122,〢,〢,er4,numeral 2 in the Suzhou numeral system 蘇州碼子|苏州...,None
123,〣,〣,san1,numeral 3 in the Suzhou numeral system 蘇州碼子|苏州...,None
124,〤,〤,si4,numeral 4 in the Suzhou numeral system 蘇州碼子|苏州...,None


In [22]:
" ".join(sorted(
    set(x for y in df_ci["simplified"].map(lambda x: [c for c in x if c not in df_zi.index]) for x in y)
))

'% 0 1 2 3 4 5 6 8 9 A B C D E F G H I J K L M N O P Q R S T U V X a c d e i l m n o r s t u w y ˋ □ ⺮ 〸 〻 龶 鿍 鿏 鿔 鿫 鿬 鿭 𪟝 𪢌 𪢠 𪣻 𪧘 𪨊 𪨶 𪩘 𪻐 𪾢 𫄧 𫄨 𫄸 𫍯 𫍲 𫍽 𫐄 𫐐 𫐓 𫐘 𫓧 𫓹 𫔍 𫔶 𫖮 𫖯 𫗦 𫗧 𫗴 𫘝 𫘦 𫘧 𫘨 𫘪 𫘬 𫚉 𫚒 𫚕 𫚖 𫚭 𫛡 𫛭 𫛶 𫞩 𫟷 𫟼 𫠆 𫠒 𫠜 𫢸 𫫇 𫮃 𫶕 𫷷 𫸩 𬀩 𬀪 𬂩 𬇕 𬉼 𬊈 𬊤 𬍛 𬒈 𬕂 𬘓 𬘘 𬘡 𬘬 𬘭 𬙂 𬙊 𬙋 𬞟 𬟁 𬟽 𬣙 𬣞 𬤊 𬧀 𬨂 𬨎 𬩽 𬪩 𬬭 𬬮 𬬸 𬬻 𬬿 𬭁 𬭊 𬭎 𬭚 𬭛 𬭤 𬭩 𬭳 𬭶 𬭸 𬭼 𬮤 𬮱 𬯀 𬯎 𬱖 𬱟 𬳵 𬳶 𬳽 𬳿 𬴊 𬶋 𬶍 𬶏 𬶐 𬶨 𬶭 𬷕 𬸚 𬸣 𬸦 𬸩 𬸪 𬺈 𭚦 𮐨 𮙋 𰦭 𰻝 𱃲 𱅒 𱇭 𱉝 𱉵 𱌶 𱌹'

In [66]:
df_ci2 = df_ci[
    df_ci["jyutping"].notna()  # note that this skips just one good word 𰻝𰻝面, but every other entry seems erroneous
    & ~df_ci["pinyin"].str.contains("xx")
    # also remove anything which cannot be written using cangjie or wubi
    & df_ci["simplified"].map(lambda x: not any(c for c in x if c not in df_zi.index))
    & df_ci["traditional"].map(lambda x: not any(c for c in x if c not in df_zi.index))
]

# TODO cedict capitalises names like Bei3 jing but we should probably mirror that for cantonese
# TODO also need to get rid of ' - ' and probably just kill all the rows with any latin characters
# TODO what to do with r5 tone?
# df_ci2 = df_ci2.assign(pinyin=df_ci2["pinyin"].str.lower())

df_ci2 = df_ci2[["simplified", "traditional", "pinyin", "jyutping", "meanings"]]

with open("../src/data/ci.json", "wt") as f:
    f.write(json.dumps(df_ci2.to_dict(orient="list"), ensure_ascii=False))
    
# with open("../src/data/ci.csv", "wt") as f:
#     f.write(json.dumps(df_ci2.to_csv(index=False), ensure_ascii=False))

In [24]:
df_ci2.head(50).tail(10)

,simplified,traditional,pinyin,jyutping,meanings
40,IP剧,IP劇,I P ju4,I P kek6,screen adaptation (TV series or movie based on...
41,JK制服,JK制服,J K zhi4 fu2,zei1 kei1 zai3 fuk6,(Japanese) high school girl’s uniform
42,K人,K人,K - ren2,kei1 jan4,(slang) to hit sb; to beat sb
43,K仔,K仔,K zai3,kei1 zai2,ketamine (slang)
44,K他命,K他命,k ta1 ming4,kei1 taa1 ming6,ketamine (C13H16ClNO) (slang)
45,K房,K房,K fang2,kei1 fong2,(slang) KTV; KTV room
46,K书,K書,K shu1,kei1 syu1,"(Tw) to cram; to study (from Taiwanese 齧書, Tai..."
47,K歌,K歌,K ge1,kei1 go1,karaoke (slang)
48,K粉,K粉,K fen3,kei1 fan2,ketamine (slang)
49,K线,K線,K xian4,kei1 sin3,candlestick (in a candlestick chart)


In [25]:
df_ci2[df_ci2["pinyin"].str.contains(" - ")]

,simplified,traditional,pinyin,jyutping,meanings
12,AA制,AA制,AA - zhi4,ei1 ei1 zai3,to split the bill; to go Dutch
42,K人,K人,K - ren2,kei1 jan4,(slang) to hit sb; to beat sb
61,OK绷,OK繃,OK - beng1,ou1 kei1 bang1,(Tw) band-aid; sticking plaster
64,O型腿,O型腿,O - xing2 tui3,ou1 jing4 teoi2,bow legs; bow-leggedness
106,e人,e人,e - ren2,e1 jan4,(slang) extroverted person
108,i人,i人,i - ren2,i jan4,(slang) introverted person


In [26]:
merged = df_hsk[["simplified", "meanings"]].merge(df_ci[["simplified", "pinyin"]], on="simplified", how="left")
merged[merged["pinyin"].isna()]

,simplified,meanings,pinyin


In [63]:
with open("../src/data/hsk.json", "wt") as f:
    f.write(json.dumps(df_hsk[["simplified", "level"]].to_dict(orient="list"), ensure_ascii=False))

In [254]:
from collections import defaultdict
import re

simps = set()
trads = set()
simp2trad = defaultdict(set)
trad2simp = defaultdict(set)
for i, row in df_py.iterrows():
    for simp, trad in zip(row.simplified, row.traditional):
        if re.match("[A-Za-z0-9]+", simp):
            continue
        simps.add(simp)
        trads.add(trad)
        if simp != trad:
            simp2trad[simp].add(trad)
            trad2simp[trad].add(simp)

In [255]:
{k:v for k,v in simp2trad.items() if len(v)>1}

{'当': {'噹', '當'},
 '系': {'係', '繫'},
 '钟': {'鍾', '鐘'},
 '复': {'復', '複', '覆'},
 '钱': {'淺', '錢'},
 '烟': {'煙', '菸'},
 '间': {'閒', '間'},
 '两': {'両', '兩'},
 '发': {'發', '髪', '髮'},
 '夹': {'夾', '挾'},
 '冲': {'沖', '衝'},
 '绝': {'絕', '絶'},
 '线': {'綫', '線'},
 '画': {'劃', '畫'},
 '台': {'枱', '檯', '臺'},
 '铺': {'舖', '鋪'},
 '为': {'為', '爲'},
 '说': {'說', '説'},
 '历': {'厤', '曆', '歷'},
 '并': {'並', '併'},
 '记': {'時', '記'},
 '墙': {'墻', '牆'},
 '干': {'乾', '幹', '榦'},
 '净': {'凈', '淨'},
 '面': {'麪', '麵'},
 '鲠': {'骾', '鯁'},
 '轰': {'揈', '轟'},
 '爷': {'爺', '耶'},
 '尽': {'儘', '盡'},
 '卫': {'衛', '衞'},
 '闲': {'閑', '閒'},
 '罗': {'囉', '羅'},
 '证': {'証', '證'},
 '鉴': {'鑑', '鑒'},
 '宝': {'寳', '寶'},
 '汇': {'匯', '彙', '滙'},
 '哗': {'嘩', '譁'},
 '饥': {'飢', '饑'},
 '厕': {'厠', '廁'},
 '厘': {'里', '釐'},
 '铲': {'剷', '鏟'},
 '须': {'須', '鬚'},
 '废': {'廢', '癈'},
 '仑': {'侖', '崙'},
 '锤': {'錘', '鎚'},
 '炼': {'煉', '鍊'},
 '阅': {'閱', '閲'},
 '迹': {'跡', '蹟'},
 '里': {'裏', '裡'},
 '尝': {'嘗', '嚐'},
 '啊': {'呵', '阿'},
 '叹': {'嘆', '歎'},
 '荡': {'盪', '蕩'},
 '获': {'獲', '穫'}

In [256]:
{k:v for k,v in trad2simp.items() if len(v)>1}

{'淺': {'浅', '钱'},
 '時': {'时', '记'},
 '網': {'纲', '网'},
 '訂': {'定', '订'},
 '囉': {'啰', '罗'},
 '屬': {'属', '𡳞'},
 '蝕': {'蚀', '食'},
 '閒': {'闲', '间', '𫔮'},
 '劃': {'划', '画'},
 '貢': {'贡', '页'},
 '麗': {'丽', '利'},
 '挾': {'夹', '挟'},
 '託': {'托', '讬'},
 '鹼': {'硷', '碱'},
 '餬': {'糊', '𫗫'},
 '齧': {'啮', '𫜩'},
 '鵰': {'雕', '𫛲'}}

In [257]:
print([c for c in trads if c not in df_cj.index])

['𥈲', 'ｓ', 'ｐ', '５', 'Ｋ', 'Ｐ', 'ｌ', 'Ｙ', 'Ｘ', 'ｋ', 'ｕ', 'ｃ', 'ｙ', 'Ｈ', 'Ｑ', 'ｇ', 'Ｕ', 'ｂ', 'ｅ', '⺕', 'ｎ', 'ｄ', 'Ｏ', '３', 'ｑ', 'ｗ', 'Ｌ', '－', 'ｏ', 'ｈ', 'Ｎ', 'ｉ', '０', 'Ｖ', 'ｔ', 'ｆ', 'Ｒ', 'Ｍ', 'Ｅ', 'Ｗ', 'Ｄ', 'Ａ', '-', 'ｒ', 'Ｉ', 'Ｓ', 'ａ', 'ｍ', '１', 'Ｔ', 'Ｂ', '６']


In [258]:
parser = CedictParser(file_path="data/cccanto-webdist.txt")
entries = parser.parse()

In [259]:
df_jp = pd.DataFrame({
    "simplified": [e.simplified for e in entries],
    "traditional": [e.traditional for e in entries],
    "pinyin": [e.pinyin for e in entries],
    "jyutping": [e.jyutping for e in entries],
    "meanings": [e.meanings for e in entries]
})
df_jp.head()

,simplified,traditional,pinyin,jyutping,meanings
0,一件还一件,一件還一件,yi1 jian4 hai2 yi2 jian4,jat1 gin6 waan4 jat1 gin6,"[a different matter, don't mix another inciden..."
1,一不小心,一不小心,yi1 bu5 xiao3 xin1,jat1 bat1 siu2 sam1,[Accidentally]
2,一世人,一世人,yi1 shi4 ren2,jat1 sai3 jan4,[the whole life (of a person); one's entire li...
3,一世英名,一世英名,yi1 shi4 ying1 ming2,jat1 sai3 jing1 ming4,[one's great reputation and fame]
4,一个,一个,yi1 ge5,jat1 go3,"[One;a, an]"


In [348]:
pycantonese.characters_to_jyutping("a")

[('a', None)]

In [261]:
df = df_py.merge(df_jp, on=["simplified", "traditional", "pinyin"], how="outer")

In [262]:
df[df["meanings_x"].ne(df["meanings_y"])]

,simplified,traditional,pinyin,meanings_x,jyutping,meanings_y
339,三,三,san1,[1. three 2. (slang) love affair (usually refe...,saam3,"[multiple times; repeatedly, # adapted from c..."
340,三,三,san1,"[multiple times; repeatedly, # adapted from c...",saam1,[1. three 2. (slang) love affair (usually refe...
435,上,上,shang4,[8. to climb 9. to get onto 10. to go up 11. t...,soeng6,"[on top, upon, above, upper, previous, first (..."
436,上,上,shang4,"[on top, upon, above, upper, previous, first (...",soeng5,[8. to climb 9. to get onto 10. to go up 11. t...
654,中,中,zhong1,"[(dialect) OK, all right, to fall into, to be ...",zung1,"[within, among, in, middle, center, while (doi..."
...,...,...,...,...,...,...
48541,龈,齦,ken3,"[gingiva, gums of the teeth, # adapted from c...",hang2,"[to gnaw, to nibble, to bite, # adapted from ..."
48567,龟,龜,gui1,"[tortoise, turtle, KangXi radical 213, a cucko...",gwan1,"[cracked, # adapted from cc-cedict]"
48568,龟,龜,gui1,"[cracked, # adapted from cc-cedict]",gwai1,"[tortoise, turtle, KangXi radical 213, a cucko..."
48590,𠺘,𠺘,lang4,"[(Cant.) hard to get along with, unfriendly, ...",long2,"[to rinse, spread thin, # adapted from cc-ced..."


In [554]:
df_ci[df_ci["simplified"].eq("𬭶")]

,simplified,traditional,pinyin,meanings,jyutping
122660,𬭶,𨭆,hei1,hassium (chemistry),hak1
